# TODO
* [ ] Agregar el ejemplo utilizando una GAN
* [ ] Correrlo con `max_iter = 5_000`
* [ ] Hacer la versión Batch
* [ ] Hacer la versión proyectada

Notebook que sirve de ejemplo para calcular el baricentro de un conjunto de datos de imágenes.

# Configuraciones iniciales

## Constantes

In [ ]:
CLEAN_LOGS = True  # If you want to clean the logs directory
SAVE_FIGS = True  # If you want to save the figures.

S_k = 10  # Batch size
REPORT_EVERY = 100  # To report at the logger
MAX_ITER = 5_000  # MAx number of iterations for the SGDW
MAX_ITER_PROJ = MAX_ITER
MAX_IMGS = 22 * 4
MAX_ITER = MAX_IMGS
REPORT_EVERY = 5  # Descomentar para debuguear
# MAX_ITER = MAX_IMGS  # Descomentar para debuguear

In [ ]:
import torch
import numpy as np
import random

SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

torch.backends.cudnn.deterministic = True

In [ ]:
from pathlib import Path
from icecream import ic


DS_NAME = "data"

CURR_PATH = Path().absolute()
ic(CURR_PATH)
BASE_PATH = CURR_PATH.parent.parent
ic(BASE_PATH)
DATA_PATH = BASE_PATH / "data"
ic(DATA_PATH)
WGAN_PATH = BASE_PATH / "wgan_gp"
ic(WGAN_PATH)
NETS_PATH = WGAN_PATH / "networks"
ic(NETS_PATH)
IMGS_PATH = CURR_PATH / "imgs" / "notebook-01"
IMGS_PATH.mkdir(parents=True, exist_ok=True)
ic(IMGS_PATH)

## Importaciones generales

In [ ]:
from bwb import utils
from bwb.sgdw import sgdw
from bwb.distributions import *
from bwb.transports import *
import torch
import bwb.plotters as plotters
import bwb.sgdw.plotters as sgdw_plotters

## Configuración del Logger

In [ ]:
import time
from pathlib import Path


# Create the logs directory
LOG_PATH = Path("logs") / f"notebook_{time.strftime('%Y%m%d_%H%M%S')}.log"
if not LOG_PATH.parent.exists():
    LOG_PATH.parent.mkdir()

# Clean the logs
if CLEAN_LOGS:
    for log_file in Path("logs").glob("*.log"):
        log_file.unlink()

In [ ]:
import logging
from bwb._logging import log_config


# Remove the handlers
log_config.remove_all_handlers()

# Define and add FileHandler
fh = logging.FileHandler(LOG_PATH)
log_config.set_default_formatter(fh)
log_config.add_handler(fh)


_log = log_config.get_logger("notebook")
log_config.set_level(level=logging.DEBUG, name="notebook")
log_config.set_level(level=logging.INFO, name="bwb.utils")
log_config.set_level(level=logging.DEBUG, name="bwb.sgdw.sgdw")
log_config.set_level(level=logging.DEBUG, name="bwb.sgdw.plotters")
log_config.set_level(level=logging.INFO, name="bwb.transports")

Esta celda es para configurar la información mostrada en el logger

In [ ]:
# Set the default options for the report
INCLUDE_OPTIONS: sgdw.ReportOptions = {
    "dt": False,
    "dt_per_iter": True,
    "iter": True,
    "step_schd": True,
    "total_time": True,
    "w_dist": False,
}

sgdw.Report.INCLUDE_OPTIONS = INCLUDE_OPTIONS

## Obtención del dataset

In [ ]:
# You can use the wrapper to transform the usual DataSet into a model set
from bwb.distributions.models import ModelDataset
import quick_torch as qt
import torchvision.transforms.v2 as T

transform_ds = T.Compose([
    T.Resize((32, 32)),
    T.ToImage(),
    T.ToDtype(torch.float32, scale=True),
    T.Lambda(lambda x: x.squeeze()),
])


def get_ds(file_path, transform=transform_ds):
    ic(file_path)
    categories = [qt.Category.FACE]
    dataset_ = qt.QuickDraw(
        root=DATA_PATH,
        categories=categories,
        transform=transform,
        download=True,
        recognized=True,
    )
    path_dataset = Path(file_path)
    dataset_.data = np.load(path_dataset).reshape(-1, 28, 28)
    dataset_.targets = np.ones(len(dataset_.data), dtype=int)
    dataset = dataset_.get_train_data()
    ic(len(dataset))

    return ModelDataset(dataset)


DS_PATH = WGAN_PATH / "dataset" / "cleaned" / f"{DS_NAME}.npy"
ds_models = get_ds(DS_PATH)

ds_models.get(0)

## Definir $\gamma_k$

Aquí se utiliza una función de la forma
\begin{equation*}
    \gamma_k = \frac{a}{(b^{1/c} + k)^c}
\end{equation*}

Con $a > 0$, $b \geq 0$ y $0.5 < c \leq 1$

La idea es que cuando $k=0$, $\gamma_0 = \frac{a}{b}$ es la proporción entre $a$ y $b$, permitiendo ajustar el valor inicial.

In [ ]:
from bwb.sgdw.utils import step_scheduler

window = 5


def test_gamma(gamma):

    for t in range(window):
        print(f"{t = :_}; {gamma(t) = :.2%}")
    print()

    init = 50
    for t in range(init, init + window):
        print(f"{t = :_}; {gamma(t) = :.2%}")
    print()

    init = 100
    for t in range(init, init + window):
        print(f"{t = :_}; {gamma(t) = :.2%}")
    print()

    init = 300
    for t in range(init, init + window):
        print(f"{t = :_}; {gamma(t) = :.2%}")
    print()

    init = 500
    for t in range(init, init + window):
        print(f"{t = :_}; {gamma(t) = :.2%}")
    print()

    init = 1_000
    for t in range(init, init + window):
        print(f"{t = :_}; {gamma(t) = :.2%}")
    print()

    init = 3_000
    for t in range(init, init + window):
        print(f"{t = :_}; {gamma(t) = :.2%}")
    print()

    init = 5_000
    for t in range(init, init + window):
        print(f"{t = :_}; {gamma(t) = :.2%}")
    print()

    # init = 10_000
    # for t in range(init, init+window):
    #     print(f"{t = :_}; {gamma(t) = :.2%}")
    # print()

    # init = 20_000
    # for t in range(init, init+window):
    #     print(f"{t = :_}; {gamma(t) = :.2%}")
    # print()

    # init = 50_000
    # for t in range(init, init+window):
    #     print(f"{t = :_}; {gamma(t) = :.2%}")
    # print()

    # init = 100_000
    # for t in range(init, init+window):
    #     print(f"{t = :_}; {gamma(t) = :.2%}")


_a = 3
_eps = 0.2
params = dict(a=_a, b=_a + 1e-2, c=0.5 + _eps)
# params = dict(a=1, b=1, c=1)

gamma = step_scheduler(**params)

test_gamma(step_scheduler(**params))

# Baricentro de imágenes

In [ ]:
pi_n = UniformDiscreteSampler[DistributionDraw]().fit(models=ds_models)

In [ ]:
dist_draw_sgdw = sgdw.DebiesedDistributionDrawSGDW(
    distr_sampler=pi_n,
    step_scheduler=step_scheduler(**params),
    batch_size=S_k,
    max_iter=MAX_ITER,
    report_every=REPORT_EVERY,
)
dist_draw_sgdw.det_params, dist_draw_sgdw.hist

In [ ]:
from bwb.sgdw.plotters import PlotterComparison

plotter_comp = PlotterComparison(
    dist_draw_sgdw, plot_every=50, n_cols=12, n_rows=2, cmap="binary_r"
)

In [ ]:
_log.info(f"Running SGD-Wasserstein with '{DS_NAME}' DS barycenter")
bar = plotter_comp.run(
    include_dict=dict(total_time=True),
)
dist_draw_sgdw.iter_params

In [ ]:
_ = plotter_comp.plot(0)